## Multiple Agent Modeling

### Model and Prompt

`langchain`은 50개 이상의 서드파티 공급업체 및 플랫폼과의 통합을 제공하며, OpenAI, Azure OpenAI, Databricks, MosaicML은 물론 허깅페이스 허브 및 오픈소스 LLM 세계와의 통합도 지원한다. 

In [11]:
import numpy as np
import pandas as pd
import openai
from openai import OpenAI
import os

with open('../../config/api.key') as file :
    lines = file.readlines()
    api_key = lines[0].strip()
    serp_api_key = lines[1].strip()
    langsmith_api_key = lines[2].strip()

openai.api_key = api_key

In [12]:
from langchain_openai import OpenAI
llm = OpenAI(
    openai_api_key = openai.api_key, 
    max_tokens = 1024
)

In [13]:
print(llm.invoke('2025년 가격이 상승할 것으로 기대되는 미국 증시의 섹터나 테마는 뭘까? 3개만 알려줘'))



1. 기��� ��터 (Technology Sector)
기��� ��터는 ���래 ��장 가능성이 ���고 ��신적인 기��들이 포함되어 있어 2025�� 가�� 상��이 기대되는 ��터입니다. 인공지능(AI), ������이터, 클라우드 ������� 등의 기��� 발전으로 ��은 기��들이 이 ��야에 ����하고 있기 때문에 ���래 ��장 가능성이 ���은 것으로 예상��니다.

2. ��경 에��지 ��터 (Environmental Energy Sector)
환경 문제가 ����도로 ����해지면서 ��경 에��지 ��터의 중요성이 ��� ��지고 있습니다. 정부의 지��적인 ��경 ���제와 기��의 ��경 �����적인 대��으로 ��경 에��지 기��들의 가치가 오를 것으로 예상��니다. ���한 ��재생 에��지 기��� 등의 발전으로 ��경 에��지 기��들의 경���력이 ��화�� 것으로 전����니다.

3. ���스���어 ��터 (Healthcare Sector)
미��의 인구 ����화 추세로 인해 ���스���어 ��터의 중요성이 ��� ��지고 있습니다. ���한 ���로나19로 인해 ���스���어 기���의 중요성이 부��되고 있습니다. 이에 ��라 ���스���어 기��들의 가치가 상��할 것으로 예상되��, ������ ��지�� ���스���어 기���이 발전하면서 이 ��야에 ���자하는 기��들의 수도 ���어�� 것으로 예상��니다. 


#### prompt template

prompt template은 언어 모델에 대한 프롬프트를 생성하는 방법을 정의하는 구성 요소이다. 여기에는 변수, placeholder, prefix, suffix, 그리고 데이터와 과업에 따라 맞춤화할 수 있는 기타 요소가 포함될 수 있다.
예를 들어, 언어 모델을 사용하여 한 언어에서 다른 언어로 번역을 생성하고 싶다고 가정해 보자. 다음과 같은 prompt template을 사용할 수 있다.

```
sentence : {sentence}
translate to {language} :
```

이 템플릿은 다음과 같이 쉽게 구현할 수 있다.

In [14]:
from langchain import PromptTemplate

template = """
문장 : {sentence}
{language}로 번역 :
"""

prompt = PromptTemplate(
    template = template,
    input_variables = ['sentence','language']
)

In [15]:
print(
    prompt.format(
        sentence = "탁자 위에 고양이가 있다.",
        language = '영어'
    )
)


문장 : 탁자 위에 고양이가 있다.
영어로 번역 :



- completion model : 텍스트 입력을 받아 텍스트 출력을 생성하는 LLM의 한 유형이다. 과업과 훈련한 데이터에 따라 일관되고 관련성 있는 방식으로 prompt를 이어가려고 노력한다. 예를 들어, 완성 모델을 프롬프트에 따라서 요약, 번역, 스토리, 코드, 가사 등을 생성할 수 있다.
- chat model : 대화 응답을 생성하도록 설계된 특수한 종류의 완성 모델이다. 메세지 목록을 입력으로 받으며, 각 메세지에는 역할(시스템, 사용자 또는 어시스턴트)과 콘텐츠가 있다. 채팅 모델은 이전 메세지와 시스템 지시를 기반으로 어시스턴트 역할에 대한 새 메세지를 생성하려고 시도한다.

완성모델과 채팅 모델의 주요 차이점은 완성 모델을 하나의 텍스트 입력을 프롬프트로 기대하는 반면, 채팅 모델은 메세지 목록을 입력으로 기대한다는 점이다.

### Data Connection

data connection은 모델에 제공하려는 추가적인 non-parametic 지식을 검색하는 데 필요한 빌딩 블록을 의미한다. 이 아이디어는 5개의 주요 블록(원천 - 로드 - 변환 - 임베드 - 저장 - 검색)으로 이뤄지는 애플리케이션에 사용자별 데이터를 통합하는 일반적인 흐름을 다루기 위한 것이다.

#### Document loaders

document loader는 CSV, file directory, HTML, JSON, markdown, PDF 등 다양한 source로부터 문서를 읽어 들이는 일을 한다. document loader는 구성된 소스로부터 데이터를 문서로 loading하기 위한 `.load` method를 노출한다. 출력은 텍스트와 관련 메타데이터를 포함하는 `Document` 객체이다.

In [16]:
dot_data = pd.read_csv(
    '../../data/dot_plot_2024_12.csv',
    index_col = 0
)

In [17]:
dot_data

,2024,2025,2026,2027,Longer run
Target rate,,,,,
4.750,NaN,NaN,NaN,NaN,NaN
4.625,4.0,NaN,NaN,NaN,NaN
4.500,NaN,NaN,NaN,NaN,NaN
4.375,15.0,1.0,NaN,NaN,NaN
4.250,NaN,NaN,NaN,NaN,NaN
4.125,NaN,3.0,NaN,NaN,NaN
4.000,NaN,NaN,NaN,NaN,NaN
3.875,NaN,10.0,3.0,2.0,1.0
3.750,NaN,NaN,NaN,NaN,1.0


In [18]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(
    file_path = '../../data/dot_plot_2024_12.csv'
)
data = loader.load()

출력은 다음과 같다. 객체가 `Document`인 것을 확인할 수 있다.

In [19]:
data

[Document(metadata={'source': '../../data/dot_plot_2024_12.csv', 'row': 0}, page_content='Target rate: 4.750\n2024: \n2025: \n2026: \n2027: \nLonger run: '),
 Document(metadata={'source': '../../data/dot_plot_2024_12.csv', 'row': 1}, page_content='Target rate: 4.625\n2024: 4\n2025: \n2026: \n2027: \nLonger run: '),
 Document(metadata={'source': '../../data/dot_plot_2024_12.csv', 'row': 2}, page_content='Target rate: 4.500\n2024: \n2025: \n2026: \n2027: \nLonger run: '),
 Document(metadata={'source': '../../data/dot_plot_2024_12.csv', 'row': 3}, page_content='Target rate: 4.375\n2024: 15\n2025: 1\n2026: \n2027: \nLonger run: '),
 Document(metadata={'source': '../../data/dot_plot_2024_12.csv', 'row': 4}, page_content='Target rate: 4.250\n2024: \n2025: \n2026: \n2027: \nLonger run: '),
 Document(metadata={'source': '../../data/dot_plot_2024_12.csv', 'row': 5}, page_content='Target rate: 4.125\n2024: \n2025: 3\n2026: \n2027: \nLonger run: '),
 Document(metadata={'source': '../../data/dot_p

#### Document Transformation

문서를 가져온 후에는 필요에 더 잘 맞게 수정하는 것이 일반적이다. 기본적인 예로, 긴 문서를 모델의 문맥 창에 맞는 작은 chunk로 나누는 것이 있다. 랭체인에는 text splitters라고 하는 다양한 사전 구축된 문서 변환기가 존재한다. 텍스트 분할기는 문맥 및 관련 정보를 보존하면서 문서를 의미론적 연관성이 있는 청크로 더 쉽게 분할한다.

텍스트 분할기를 사용하면 텍스트를 분할하는 방법과 청크의 길이를 측정하는 방법을 결정할 수 있다. 예를 들어, 글자(character) 수준에서 작동하는 `RecursiveCharacterTextSplitter` module을 사용하여 문서를 분할해 보자.

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('../../data/Artificial Intelligence in the Financial System.txt') as file :
    text_file = file.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = len
)

texts = text_splitter.create_documents([text_file])

여기서 `chunk_size`는 각 청크의 글자 수를 나타내고 `chunk_overlap`은 연속된 청크 간에 겹치는 글자 수를 나타낸다. 출력은 다음과 같다

In [21]:
print(texts[0])
print(texts[1])
print(texts[2])

page_content='Discussions of artificial intelligence (AI) inevitably center on two main points: risks and'
page_content='points: risks and benefits.1 Both of these can be frustratingly vague and amorphous. Proponents of'
page_content='Proponents of AI project its widespread adoption will be as momentous as the industrial'


#### Text Embedding Model

embedding은 nonparametic 지식을 LLM에 통합하는 핵심 단계이다. 실제로 embedding이 vector DB에 제대로 저장되면 사용자 쿼리의 거리를 측정할 수 있는 비모수적 지식이 된다.

임베딩을 시작하려면 임베딩 모델이 필요한데, 랭체인은 비모수적 지식과 사용자 쿼리의 임베딩을 각각 처리하는 두 가지 주요 모듈로 구성된 `Embedding` 클래스를 제공한다.

In [22]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()
os.environ['OPENAI_API_KEY'] = openai.api_key

In [24]:
embedding_model = OpenAIEmbeddings(model = 'text-embedding-3-small')
embeddings = embedding_model.embed_documents(
    [
        "Good morning!",
        "Good afternoon!",
        'Oh, hello!',
        'I want to report an accident',
        'Sorry to hear that, May I ask your name?',
        'Sure, Mario Rossi.',
    ]
)

print('<embedded documents>')
print(
    f"Number of vector : {len(embeddings)} \nDimension of vector : {len(embeddings[0])}"
)

<embedded documents>
Number of vector : 6 
Dimension of vector : 1536


문서 질의가 모두 임베딩된 후에 할 일은 두 요소 간의 유사도를 계산하고 문서 임베딩에서 가장 적합한 정보를 검색하는 것이다.

In [26]:
embedded_query = embedding_model.embed_query(
    "What was the name mentioned in the conversation?"
)

In [27]:
print('<questionare>')
print(f"Dimension of ter vector : {len(embeddings[0])}")
print(f"Sample of the first 5 elements of the vector : {embedded_query[:5]}")

<questionare>
Dimension of ter vector : 1536
Sample of the first 5 elements of the vector : [-0.010684116743505001, -0.010173137299716473, -0.0019674645736813545, 0.023056013509631157, -0.02686513401567936]


#### Vector Database

Vector DB는 임베딩을 사용하여 텍스트, 이미지, 오디오 또는 비디오와 같은 비정형 데이터를 저장하고 검색할 수 있는 데이터베이스의 한 유형이다. 임베딩을 사용하면 벡터 저장소는 빠르고 정확한 유사도 검색, 즉 주어진 쿼리에 가장 관련성이 높은 데이터를 사용할 수 있다. 여기서는 고밀도 벡터의 효율적인 유사도 검색 및 클러스터링을 위해 META AI Research에서 개발한 FAISS Vector storage를 사용해 보자.

In [28]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from dotenv import load_dotenv

load_dotenv()
os.environ['OPENAI_API_KEY'] = openai.api_key

In [29]:
raw_documents = TextLoader('../../data/Artificial Intelligence in the Financial System.txt').load()
text_splitter = CharacterTextSplitter(
    chunk_size = 1100,
    chunk_overlap = 0,
    separator = '\n'
)
documents = np.array(text_splitter.split_documents(raw_documents))

In [30]:
texts = [doc.page_content for doc in documents]

In [31]:
db = FAISS.from_documents(texts, OpenAIEmbeddings())

/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_90151/68036774.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = FAISS.from_documents(texts, OpenAIEmbeddings())


AttributeError: 'str' object has no attribute 'page_content'

이제 비모수적 지식을 임베딩하고 저장했으니, 코사인 유사도를 측정값으로 사용하여 가장 유사한 텍스트 청크를 검색하는 데 사용할 수 있도록 사용자의 쿼리도 임베딩해 보자

In [32]:
query = "How does AI affect the financial system?"
docs = db.similarity_search(query)
print(docs[0].page_content)

NameError: name 'db' is not defined

출력은 질문에 대한 답을 포함할 가능성이 더 높은 텍스트 조각이다. 엔드 투 엔드 시나리오에서는 대화형 응답을 생성하기 위한 LLM의 문맥으로 사용된다.

#### Retrievers

검색기는 자연어 질문이나 키워드와 같은 비정형 쿼리와 관련된 문서를 반환할 수 있는 랭체인의 구성 요소이다. 검색기는 문서 자체를 저장할 필요 없이 source에서 문서를 검색하기만 하면 된다. 검색기는 키워드 매칭, 의미론적 검색, 순위 알고리즘 등 다양한 방법을 사용해 관련 문서를 찾을 수 있다.

검색기와 벡터 저장소의 차이점은 검색기가 벡터 저장소보다 더 일반적이고 유연하다는 점이다. 검색기는 모든 방법을 사용해 관련 문서를 찾을 수 있는 반면, 벡터 저장소는 임베딩과 유사도 메트릭에 의존한다. 또한 검색기는 웹 페이지, 데이터베이스, 파일 등 다양한 문서 소스를 사용할 수 있는 반면, 벡터 저장소는 데이터 자체를 저장해야 한다.

그러나 데이터가 벡터 저장소에 의해 임베딩되고 색인된 경우, 벡터 저장소를 검색기의 백본으로 사용할 수도 있다. 이 경우 검색기는 벡터 저장소를 사용하여 임베딩된 데이터에 대해 유사도 검색을 수행하고 가장 관련성이 높은 문서를 반환할 수 있다. 이것은 랭체인의 주요 검색기 유형 중 하나이며, 벡터 저장소 검색기라고 불린다.

예를 들어서, 이전에 초기화했던 FAISS 벡터 저장소를 고려하고 그 위에 검색기를 마운팅해 보자.

In [33]:
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI

retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(
    llm = OpenAI(), 
    chain_type = 'stuff',
    retriever = retriever
)
query = "How does AI affect the financial system?"
qa.invoke(query)

NameError: name 'db' is not defined

### Memories

LLM기반 애플리케이션에서 기억은 애플리케이션이 장단기적으로 사용자 상호 작용에 대한 참조를 유지할 수 있게 해 준다. Chat GPT를 사용할 때를 떠올려 보자. 사용자는 Chat GPT와 상호 작용하는 동안 모델에 명시적으로 말하지 않고도 이전 상호 작용을 참조해 후속 질문을 할 수 있다. 또한 모든 대화는 타래에 저장되므로, 이전 대화를 이어서 하고 싶을 경우 모든 문맥을 Chat GPT에 제공할 필요 없이 타래를 다시 열기만 하면 된다. 이는 사용자의 상호 작용을 기억 변수에 저장하고 후속 질문을 처리하는 동안 이 기억을 문맥으로 사용할 수 있는 Chat GPT의 기능 덕분에 가능하다.

langchain은 애플리케이션 내에서 기억 시스템을 설계할 수 있는 여러 모듈을 제공해 읽기, 쓰기 기능을 모두 지원한다. 기억 시스템에서 해야 할 첫 번째 단계는 인간과의 상호 작용을 실제로 어딘가에 저장하는 것이다. 이를 위해 다양한 기억 시스템을 Redis, Cassandra, Postgres등 다양한 서드파티와 통합해 활용할 수 있다. 그 다음, 기억 시스템에 질의하는 방법을 정의할 때 활용할 수 있는 다양한 기억 유형이 있다.

- conversation buffer memory : 랭체인에서 사용할 수 있는 '단순한 기본형' 기억 유형이다. 이를 통해 채팅 메세지를 저장하고 변수에 추출할 수 있다.
- conversation buffer window memory : 대화 버퍼 기억과 같되, 시간이 지남에 따라서 긴 대화 기록을 관리할 수 있도록 최대 K회까지 상호 작용을 기억하는 sliding window를 둔다는 차이가 있다.
- entity memory : 개체 기억은 언어 모델이 대화에서 측정 개체에 관해 주어진 사실을 기억할 수 있도록 하는 랭체인의 기능이다. LLM을 사용해 입력 테스트에서 개체에 대한 정보를 추출하는 방식으로 작동하며, 추출된 사실을 기억 저장소에 저장해 시간이 지남에 따라서 해당 개체에 대한 지식을 쌓는다. 언어 모델은 개체에 대한 새로운 정보를 기억하거나 학습해야 할 때마다 기억 저장소에 액세스하고 업데이트할 수 있다.
- conversation knowledge graph memory : 지식 그래프를 사용해 기억을 생성한다. 여기서 지식 그래프는 그래프 구조로 지식을 표현하고 정리하는 방식을 의미한다. 대화 지식 그래프 기억을 사용해, 각 대화의 입출력을 지식으로 저장한 다음, 이를 사용해 현재 상황에 따라 관련성 있고 일관성있는 답변을 생성할 수 있다. 또한, 지식 그래프에서 질의해 현재 개체 또는 대화 이력을 가져올 수도 있다.
- conversation summary memory : 시간이 지남에 따름 대화 요약을 생성한다. 저장할 대화가 길어질 때 매우 유용하다.
- conversation summary buffer memory : 버퍼 기억과 대화 요약 기억의 아이디어를 결합한 것이다. 최근 대화에 대한 버처를 기억하되, 오래된 대화를 완전히 삭제하지 않고 요약을 만들어서 함께 사용한다.
- conversation token buffer memory : 대화 요약 버퍼 기억과 유사하되, 상호 작용 요약 시작 시점을 결정하기 위해 상호작용의 횟수가 아닌 토큰 길이를 기준으로 한다는 차이가 있다.
- vector store-backed memory : 이 유형의 기억은 앞서 다룬 임베딩과 벡터 저장소의 개념을 활용한다. 상호 작용을 백터로 저장한 다음 검색기를 사용해 질의할 때마다 가장 유사도가 높은 상위 K개의 텍스트를 검색한다는 점에서 이전의 모든 기억과 다르다.

In [37]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI

memory = ConversationSummaryMemory(llm = OpenAI(temperature = 0))
memory.save_context(
    {'input': '2025년에 미국 주식 시장 소형주 투자를 하고 싶은데 투자할만한 종목이 있을까? 퀀텀컴퓨팅 및 AI와 관련된 종목 위주로 했으면 좋겠어.'},
    {'output': '2025년 투자할 만한 미국 소형주에는 Spectral AI(ticker : MDAI)가 있습니다.'}
)
memory.load_memory_variables({}) # 저장된 타래로부터 불러온다

{'history': '\nThe human asks for investment advice for the US small-cap market in 2025, specifically looking for stocks related to quantum computing and AI. The AI suggests Spectral AI (ticker: MDAI) as a potential option.'}

메모리는 초기화한 OpenAI LLM을 활용해 대화를 요약하였다. 애플리케이션 내에서 어떤 기억을 사용할지 정해진 바는 없지만, 특정 기억이 특히 적합한 몇 가지 시나리오가 있다. 예를 들어, 지식 그래프 기억은 대규모의 다양한 데이터 말뭉치에서 정보에 액세스하고 의미론적 관게에 기반해 응답을 생성해야 하는 애플리케이션에 유용하며, 대화 요약 버퍼 기억은 여러 차례에 걸쳐 일관된 문맥을 유지하면서 이전 대화 이력을 압축 및 요약할 수 있는 대화형 에이전트를 만드는 데 적합할 수 있다.

### Chain

chain은 일련의 작동과 LLM 호출을 미리 정해둔 것이다. 체인을 활용하면 LLM을 서로 또는 다른 컴포넌트와 결합해야 하는 복잡한 애플리케이션을 쉽게 구축할 수 있다. 

랭체인은 시작하기 위한 네 가지 주요 유형의 체인을 제공한다.

#### LLM Chain

가장 일반적인 유형의 체인이다. prompt template, LLM, output parser로 구성된다. 여기서 output parser는 언어 모델 응답을 구조화하는 데 도움이 되는 컴포넌트이다.

In [38]:
from langchain import PromptTemplate

template = """
sentence : {sentence}
translate to {language}
"""
prompt = PromptTemplate(
    template = template,
    input_variables = ['sentence','language']
)

이 체인은 여러 입력 변수를 받아서 `PromptTemplate`을 사용해 프롬프트 형식으로 formatting을 하고 모델에 전달한다. 이제 이를 LLMChain에 넣어 보자

In [39]:
from langchain.chains import LLMChain
from langchain_openai import OpenAI

llm = OpenAI(temperature = 0)
llm_chain = LLMChain(prompt = prompt, llm = llm)
llm_chain.predict(
    sentence = "지난 해 애플 주식을 10주 샀는데 20%가 상승했어요.",
    language = '영어'
)

/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_90151/862837946.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt = prompt, llm = llm)


'Last year, I bought 10 shares of Apple stock and it rose by 20%.'

In [40]:
# LLMChain deprecation 해결

from langchain_openai import OpenAI
from langchain import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnablePassthrough

template = """
sentence : {sentence}
translate to {language}
"""
prompt = PromptTemplate(
    template = template, 
    input_variables = ["sentence", "language"]
)

llm = OpenAI(temperature = 0)

output_parser = StrOutputParser()

chain = RunnableSequence(
    {
        "sentence": RunnablePassthrough(),
        "language": RunnablePassthrough()
    }
    | prompt
    | llm
    | output_parser
)

result = chain.invoke({
    'sentence' : "지난 해 애플 주식을 10주 샀는데 20%가 상승했어요.",
    'language' : '영어'
})
print(result)

Last year, I bought 10 shares of Apple stock and it rose by 20%.


#### RouterChain

특정 조건에 따라 입력 변수를 다른 체인으로 라우팅할 수 있는 체인 유형이다. 조건 값을 반환하는 함수나 표현식으로 지정할 수 있다. 조건이 충족되지 않는 경우 사용할 기본 체인을 지정할 수도 있다. 

예를 들어서 이 체인을 사용하여 여행 일정 계획이나 레스토랑 예약과 같은 다양한 유형의 요청을 처리할 수 있는 챗봇을 만들 수 있다. 이 목표를 달성하기 위해 사용자의 쿼리 유형에 따라 두 가지 프롬프트를 차별화할 수 있다.

In [41]:
portfolio_template = """
당신은 전문 투자 조언가입니다.
당신은 고객이 제시한 종목 중 상승 가능성이 가장 큰 종목 혹은 가장 성장가능성이 큰 종목을 찾도록 도와줍니다.
당신은 고객의 선호에 따라 2025년 기준으로 최선의 포트폴리오를 구성하도록 도움을 줍니다.

여기에 질문이 있습니다 :
{input}
"""

risk_manager_template = """
당신은 포트폴리오 리스크를 관리해주는 전문 리스크 매니저입니다.
당신은 고객이 보유중인 포트폴리오의 잠재적인 시장 위험 및 신용 위험, 운영 위험 등을 평가합니다.
여기서 고객이 얼마나 리스크를 감당할 수 있는지 고려해야 합니다.

여기 질문이 있습니다 :
{input}
"""

`RouterChain` 덕분에 사용자의 쿼리에 따라 다른 프롬프트를 활성화할 수 있는 체인을 구축할 수 있다. 체인이 두 가지 다른 사용자의 쿼리에 어떻게 반응하는지에 대한 샘플 출력을 볼 수 있다.

In [42]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

prompt_infos = [
    {
        "name": "포트폴리오 구성",
        "description": "주식 종목 포트폴리오 구성을 돕습니다",
        "prompt_template": portfolio_template,
    },
    {
        "name": "리스크 관리",
        "description": "고객의 포트폴리오 위험 관리를 도와줍니다",
        "prompt_template": risk_manager_template,
    },
]

In [43]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm = llm, output_key = "text")

/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_90151/812518829.py:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  default_chain = ConversationChain(llm = llm, output_key = "text")
/Users/junghunlee/anaconda3/envs/langchain/lib/python3.11/site-packages/pydantic/main.py:214: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [44]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [45]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_90151/1641175374.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(


In [46]:
print(
    chain.run('저는 현재 미국 NASDAQ에 상장되어 있는 AAPL, MSFT, WMT, JPM, SPY를 투자할까 고민중이에요.')
)

/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_90151/4024610952.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run('저는 현재 미국 NASDAQ에 상장되어 있는 AAPL, MSFT, WMT, JPM, SPY를 투자할까 고민중이에요.')




> Entering new MultiPromptChain chain...
���트���리오 구성: {'input': '��는 ��재 ����� NASDAQ에 상장되어 있는 AAPL, MSFT, WMT, JPM, SPY를 ���자할�� ������이에요.'}

ValueError: Received invalid destination chain name '���트���리오 구성'

#### Sequential Chain

여러 개의 체인을 순서대로 실행할 수 있는 체인 유형이다. 체인의 순서와 출력을 다음 체인에 전달하는 방법을 지정할 수 있다. 순차적 체인의 가장 단순한 모듈로, 기본적으로 한 체인의 출력을 다음 체인의 입력으로 사용한다. 그러나 더 복잡한 모듈을 사용하여 체인 간의 입출력을 보다 유연하게 설정할 수도 있다.

예를 들어, 주어진 주제에 대한 요약을 먼저 생성한 다음 이를 다른 언어로 번역하는 AI 시스템을 생각해 보자. 이를 위해 먼저 두 개의 체인을 생성한다.

In [47]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature = 0.7)
template = """
당신은 요약 전문가입니다. {text}에 관한 요약을 생성하세요.
본문 : 
"""
prompt_template = PromptTemplate(
    input_variables = ['text'],
    template = template
)
summary_chain = LLMChain(
    llm = llm,
    prompt = prompt_template
)

In [48]:
template = """
당신은 번역가입니다. 주어진 텍스트 입력을 {language}로 번역하세요.
번역 :
"""
prompt_template = PromptTemplate(
    input_variables = ['language'],
    template = template
)
translator_chain = LLMChain(
    llm = llm,
    prompt = prompt_template
)

이제 `SimpleSequentialChain` module을 사용해 이들을 결합해 보자

In [49]:
text_demo = """
Due to evolving legal landscape & changes in the framework of administrative law, Federal Reserve Board will soon seek public comment on significant changes to improve transparency of bank stress tests & reduce volatility of resulting capital requirements

In view of the evolving legal landscape, the Federal Reserve Board will soon seek public comment on significant changes to improve the transparency of its bank stress tests and to reduce the volatility of resulting capital buffer requirements.

The Board's stress test evaluates the resilience of large banks by estimating their losses, revenue, and capital levels under a hypothetical recession scenario that changes each year. Capital acts as a cushion to absorb losses and allows banks to continue lending to households and businesses even during a recession. Since its inception over 15 years ago, large banks in the stress test have more than doubled their capital levels, an increase of more than $1 trillion.

The Board intends to propose changes that include, but are not limited to: disclosing and seeking public comment on all of the models that determine the hypothetical losses and revenue of banks under stress; averaging results over two years to reduce the year-over-year changes in the capital requirements that result from the stress test; and ensuring that the public can comment on the hypothetical scenarios used annually for the test, before the scenarios are finalized. These proposed changes are not designed to materially affect overall capital requirements.

The framework of administrative law has changed significantly in recent years. The Board analyzed the current stress test in view of the evolving legal landscape and determined to modify the test in important respects to improve its resiliency.

The Board will continue its exploratory analysis, which assesses additional risks to the banking system in ways that are separate from the stress test. The analysis would be used to inform bank supervision and financial stability assessments. It will continue to be disclosed in aggregate and not affect bank capital requirements.

For the 2025 stress test, the Board plans to take immediate steps to reduce the volatility of the results and begin to improve model transparency. The Board intends to begin the public comment process on its comprehensive changes to the stress test during the early part of 2025.

For media inquiries, please email media@frb.gov or call 202-452-2955.
"""

In [50]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(
    chains = [summary_chain, translator_chain],
    verbose = True
)
translated_summary = overall_chain.run(text_demo)



> Entering new SimpleSequentialChain chain...
미�� 연����비제도이사회(Federal Reserve Board)는 ��적 ���의 변화와 ���정��의 변화를 ����하여 ����� ��트��스 ���스트의 ���명성을 ���상시���고 결과로 나오는 자�� 요구��의 변동성을 ��이기 위한 중요한 변경사��에 대한 ��개 ����을 �� ���할 예정입니다.

이사회의 ��트��스 ���스트는 ���� 변화하는 가상의 ���경기 시나리오에서 ����, 수�� �� 자�� 수��을 추정하여 대�� �����의 내구성을 ��가합니다. 자��은 ����을 ���수하는 ��할을 하�� �����이 ���경기 기간에도 가��와 기��에 대출을 계��할 수 있게 ��니다. 15�� 이상 전에 시작
You are a translator. Federal Reserve Board, which is the monetary policy authority in the United States, is planning to make important changes to maintain the reputation of the financial system and to overcome the volatility of demand for resources resulting from changes in economic conditions. As part of this, the Board's stress tests estimate losses and capital needs in a hypothetical scenario of changing economic conditions to assess the resilience of the financial system. This allows banks to make plans to maintain their stability and continue lending during times of economi

#### Transformation Chain

일부 함수나 표현식을 사용해 입력 변수나 다른 체인의 출력을 변환할 수 있는 체인 유형이다. 입력 또는 출력을 인수로 받아 새 값을 반환하는 함수로 변환을 지정할 수 있으며, 체인의 출력 형식을 지정할 수도 있다.

### Agent

에이전트는 LLM기반 애플리케이션 내에서 의사 결정을 내리는 주체이다. 에이전트는 일련의 도구에 액세스할 수 있으며 사용자 입력과 상황에 따라 어떤 도구를 호출할지 결정할 수 있다. Agent는 동적이고 적응력이 뛰어나므로 상황이나 목표에 따라 작동을 변경하거나 조정할 수 있다. 실제로 체인에서는 작동 순서가 하드코딩돼 있지만 에이전트에서는 올바른 순서로 올바른 작동을 계획하고 실행하는 것을 목표로 LLM이 추론 엔진으로 사용된다.

에이전트에 대해 이야기할 때 핵심 개념은 도구의 개념이다. 실제로, 에이전트가 사용자의 쿼리를 처리하기 위해 모든 적절한 행동을 계획하는 데 능숙하지만 정보나 실행력이 부족하여 실제로 실행할 수 없다면 어떻게 될까? 예를 들어서, 웹 검색을 통해 질문에 답을 할 수 있는 에이전트를 구축하고 싶다고 해 보자. 에이전트 자체로는 웹에 액세스할 수 없으므로, 도구로 내용을 제공해야 한다. 이를 위해서는 `Langchain`에서 제공하는 `SerpAPI`를 사용한다.

In [51]:
from langchain import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain_openai import OpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
import os
from dotenv import load_dotenv

load_dotenv()
os.environ['SERPAPI_API_KEY'] = serp_api_key

In [52]:
search = SerpAPIWrapper()
tools = [
    Tool.from_function(
        func = search.run,
        name = 'Search',
        description = '현재 이벤트에 관해 질문할 때 유용함'
    )
]

agent = initialize_agent(
    tools,
    llm = OpenAI(),
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True,
)

agent.invoke('팔란티어(PLTR)가 NASDAQ100에 편입된 날짜는?')

/var/folders/1q/pl9tj55n57s9jg28npxw61n80000gn/T/ipykernel_90151/3225516955.py:10: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(




> Entering new AgentExecutor chain...
 이벤트에 관해 가장 잘 알 수 있는 정보를 찾기 위해 노력해야함
Action: Search
Action Input: "팔란티어 NASDAQ100"
Observation: {'type': 'finance_results', 'title': 'Palantir Technologies Inc', 'exchange': 'NASDAQ', 'stock': ' PLTR', 'currency': 'USD', 'price': 110.85, 'previous_close': 111.28}
Thought: 해당 정보를 통해 팔란티어가 NASDAQ100에 편입된 날짜를 알 수 있음
Action: Search
Action Input: "팔란티어 NASDAQ100 편입 날짜"
Observation: 지수: S&P 500에 2020년 12월 21일 편입. 주가 상승률: 발표 이후 약 70% 상승. 이는 시장의 과도한 기대감과 테슬라의 높은 성장성을 반영한 예외적인 사례입니다. 지수: 나스닥 100에 2020년 4월 편입.
Thought: 팔란티어가 S&P 500과 NASDAQ100에 각각 언제 편입되었는지 알 수 있음
Action: Search
Action Input: "팔란티어 S&P 500 편입 날짜"
Observation: S&P500지수 운영 주체인 S&P 글로벌은 전 거래일인 지난 6일 장 마감 후 "오는 23일부터 팔란티어를 S&P500지수 구성 종목으로 편입한다"고 발표했다.
Thought: 팔란티어가 S&P 500에 편입된 날짜는 2021년 9월 23일임
Final Answer: 2021년 9월 23일

> Finished chain.


{'input': '팔란티어(PLTR)가 NASDAQ100에 편입된 날짜는?', 'output': '2021년 9월 23일'}

Agent를 초기화할 때 에이전트의 유형을 `ZERO_SHOT_REACT_DESCRIPTION`으로 설정하였는데, 이것은 우리가 선택할 수 있는 구성 중 하나이며, 특히 에이전트가 ReAct 접근 방식을 사용해 도구의 설명만으로 어떤 도구를 선택할지 결정하도록 구성한다. 이 구성 외에도 langchain은 다음과 같은 유형의 에이전트를 제공한다.

#### Structured input ReAct 

정형 입력 데이터를 기반으로 자연어 응답을 생성하기 위해 ReAct 프레임워크를 사용하는 에이전트 유형이다. 에이전트는 테이블, 목록, 또는 key-value 쌍과 같은 다양한 유형의 입력 데이터를 처리할 수 있다. 에이전트는 언어 모델과 프롬프트를 사용하여 유익하고 간결하며 일관성 있는 응답을 생성한다.

#### Open AI functions

Open AI 함수 API를 사용해 OpenAI의 다양한 언어 모델 및 도구에 액세스할 수 있는 에이전트 유형이다. 언어 모델과 프롬프트를 사용해 OpenAI 함수 API에 대한 요성을 생성하고 응답을 가져온다.

#### Conversation

언어 모형을 사용해 사용자와 자연어 대화를 하는 에이전트 유형이다. 에이전트는 채팅, 질문 답변 또는 과업 완료와 같은 다양한 유형의 대화형 과업을 처리할 수 있다. 에이전트는 언어 모델과 프롬프트를 사용해 관련성 있고 유창하며 매력적인 응답을 생성한다.

#### Self talk with search

언어 모델을 사용하여 스스로 질문을 생성한 다음, 웹에서 답을 검색하는 에이전트 유형이다. 에이전트는 이 기술을 사용하여 새로운 정보를 학습하거나 자신의 지식을 테스트할 수 있다.

#### ReAct document store 

데이터베이스에 저장된 문서를 기반으로 자연어 응답을 생성하기 위해 ReAct 프레임워크를 사용하는 에이전트 유형이다. 에이전트는 뉴스 기사,블로그 게시물 또는 연구 논문과 같은 다양한 유형의 문서를 처리할 수 있다.

#### Plan & execute agents

언어 모델을 사용해 사용자의 입력과 목표에 따라서 수행할 일련의 행동을 선택하는 실험적인 에이전트 유형이다. 에이전트는 선택한 행동을 실행하기 위해 다양한 도구나 모델을 사용할 수 있다. 에이전트는 언어 모델과 프롬프트를 사용하여 계획, 행동 들을 생성한 다음 `AgentExecuter`를 사용하여 이를 실행한다.

Langchain Agent는 LLM이 외부 세계와 상호 작용하게 할 때마다 중추적인 역할을 한다. 또한 에이전트가 LLM을 검색하고 응답을 생성할 뿐만 아니라, 추론 엔진으로 활용하여 최적화된 행동 시퀀스를 계획하는 방법도 굉장히 흥미롭다.